In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#导入数据

card_feature=pd.read_csv('input/card_test_f.csv')
#card_feature.head()

In [2]:
#导入数据

card_feature=pd.read_csv('input/card_test_f_2.csv')
#card_feature.head()

In [4]:
#card_feature.info()

In [5]:
#删除无用列

card_feature.drop(['consume','where','how','time','hour','month','weekday'],axis=1,inplace=True)

In [6]:
card_feature.head()

,id,amount,remainder,date,year,hour_00_01,hour_06_07,hour_08_09,hour_11_13,hour_17_21,...,hospital,bus,landry,shower,market,canteen,f_ten_canteen,other_canteen,f_ten_place,other_place
0,16894,0.10,432.76,09-01-2013,2013,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,10124,0.60,113.26,09-01-2013,2013,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,5016,0.01,134.37,09-01-2013,2013,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,776,1.10,61.40,09-01-2013,2013,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,13112,0.90,231.40,09-01-2013,2013,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [7]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(card_feature.columns)[5:]

#count_columns
count_pivot=card_feature.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)

In [8]:
#根据id聚合求解各个特征均值(各特征消费次数占比)

a=['id','date','year']
mean_columns=[x for x in list(card_feature.columns) if x not in a]
mean_pivot=card_feature.pivot_table(index='id',values=mean_columns,aggfunc=np.mean)

#对列名重命名
mean_pivot.columns=[x+'_mean' for x in list(mean_pivot.columns)]

#连接两表
card_feature_f=mean_pivot.join(count_pivot)

#计算消费总额与余额总额
card_feature_s=card_feature.groupby('id')['amount','remainder'].sum()
card_feature_f=card_feature_f.join(card_feature_s)

In [9]:
#获得id关于每一列的消费总额

def get_colunms_sum(df):
    a=list(df.columns)[5:]
    columns_sum=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_sum.columns=['id_count']
    for x in range(len(a)):
        if x==0:
            columns_sum_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].sum())
            columns_sum_x.columns=[a[x]+'_amount_sum']
            columns_sum=columns_sum.join(columns_sum_x)
        else:
            columns_sum_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].sum())
            columns_sum_x.columns=[a[x]+'_amount_sum']
            columns_sum=columns_sum.join(columns_sum_x)
            x+=1
    columns_sum.drop('id_count',axis=1,inplace=True)
    return columns_sum

#运用定义的函数
card_columns_sum=get_colunms_sum(card_feature)

#连接总表
card_feature_f=card_feature_f.join(card_columns_sum)

In [10]:
#定义id关于每一列的消费均值(每次)

def get_colunms_mean(df_sum,df_count):
    a=list(df_count.columns)
    columns_mean=DataFrame(df_count[a[0]])#初始化表格大小
    columns_mean.columns=['id_count']
    for x in range(len(a)):
        columns_count_x=df_count[a[x]].values
        columns_sum_x=df_sum[a[x]+'_amount_sum'].values
        columns_mean[a[x]+'_amount_mean']=np.divide.reduce([columns_sum_x,columns_count_x],axis=0)
        x+=1
    columns_mean.drop('id_count',axis=1,inplace=True)
    return columns_mean

#运用定义的函数
card_columns_mean=get_colunms_mean(card_columns_sum,count_pivot)

#连接总表
card_feature_f=card_feature_f.join(card_columns_mean)

In [11]:
#定义id关于每一列的消费额占总额的比例

def get_colunms_amount_rate(df_sum,df):
    a=list(df_sum.columns)
    columns_amount_rate=DataFrame(df_sum[a[0]])#初始化表格大小
    columns_amount_rate.columns=['id_count']
    amount_sum=df['amount'].values
    for x in range(len(a)):
        columns_amountsum_x=df_sum[a[x]].values
        columns_amount_rate[a[x]+'_rate']=np.divide.reduce([columns_amountsum_x,amount_sum],axis=0)
        x+=1
    columns_amount_rate.drop('id_count',axis=1,inplace=True)
    return columns_amount_rate

#运用定义的函数
card_columns_amount_rate=get_colunms_amount_rate(card_columns_sum,card_feature_f)

#连接总表
card_feature_f=card_feature_f.join(card_columns_amount_rate)

In [12]:
#获得id关于每一列的消费的最大值

def get_colunms_amount_max(df):
    a=list(df.columns)[5:]
    columns_amount_max=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_max.columns=['id_count']
    for x in range(len(a)):
        if x==0:
            columns_max_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].max())
            columns_max_x.columns=[a[x]+'_amount_max']
            columns_amount_max=columns_amount_max.join(columns_max_x)
        else:
            columns_max_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].max())
            columns_max_x.columns=[a[x]+'_amount_max']
            columns_amount_max=columns_amount_max.join(columns_max_x)
            x+=1
    columns_amount_max.drop('id_count',axis=1,inplace=True)
    return columns_amount_max

#运用定义的函数
columns_amount_max=get_colunms_amount_max(card_feature)

#连接总表
card_feature_f=card_feature_f.join(columns_amount_max)

In [13]:
#获得id关于每一列的消费的最小值

def get_colunms_amount_min(df):
    a=list(df.columns)[5:]
    columns_amount_min=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_min.columns=['id_count']
    for x in range(len(a)):
        if x==0:
            columns_min_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].min())
            columns_min_x.columns=[a[x]+'_amount_min']
            columns_amount_min=columns_amount_min.join(columns_min_x)
        else:
            columns_min_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].min())
            columns_min_x.columns=[a[x]+'_amount_min']
            columns_amount_min=columns_amount_min.join(columns_min_x)
            x+=1
    columns_amount_min.drop('id_count',axis=1,inplace=True)
    return columns_amount_min

#运用定义的函数
columns_amount_min=get_colunms_amount_min(card_feature)

#连接总表
card_feature_f=card_feature_f.join(columns_amount_min)

In [14]:
#获得id关于每一列的消费的标准方差

def get_colunms_amount_std(df):
    a=list(df.columns)[5:]
    columns_amount_std=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_std.columns=['id_count']
    for x in range(len(a)):
        if x==0:
            columns_std_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].std())
            columns_std_x.columns=[a[x]+'_amount_std']
            columns_amount_std=columns_amount_std.join(columns_std_x)
        else:
            columns_std_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].std())
            columns_std_x.columns=[a[x]+'_amount_std']
            columns_amount_std=columns_amount_std.join(columns_std_x)
            x+=1
    columns_amount_std.drop('id_count',axis=1,inplace=True)
    return columns_amount_std

#运用定义的函数
columns_amount_std=get_colunms_amount_std(card_feature)

#连接总表
card_feature_f=card_feature_f.join(columns_amount_std)

In [15]:
#获得id关于每一列的消费的中位数

def get_colunms_amount_median(df):
    a=list(df.columns)[5:]
    columns_amount_median=DataFrame(df.groupby('id')['id'].count())#初始化表格大小
    columns_amount_median.columns=['id_count']
    for x in range(len(a)):
        if x==0:
            columns_median_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].median())
            columns_median_x.columns=[a[x]+'_amount_median']
            columns_amount_median=columns_amount_median.join(columns_median_x)
        else:
            columns_median_x=DataFrame(df.loc[df[a[x]]==1].groupby('id')['amount'].median())
            columns_median_x.columns=[a[x]+'_amount_median']
            columns_amount_median=columns_amount_median.join(columns_median_x)
            x+=1
    columns_amount_median.drop('id_count',axis=1,inplace=True)
    return columns_amount_median

#运用定义的函数
columns_amount_median=get_colunms_amount_median(card_feature)

#连接总表
card_feature_f=card_feature_f.join(columns_amount_median)

# 第二次处理

In [ ]:
#导出数据
#card_feature_f.to_csv('input/card_feature_f1.csv',index='id')

#导入数据
card_feature_f=pd.read_csv('input/card_feature_f1.csv',index_col='id')

In [16]:
#计算单笔消费额的最大、最小值、中位数、方差
a='amount'
card_feature_f[a+'_max']=card_feature.groupby('id')[a].max()
card_feature_f[a+'_min']=card_feature.groupby('id')[a].min()
card_feature_f[a+'_median']=card_feature.groupby('id')[a].median()
card_feature_f[a+'_std']=card_feature.groupby('id')[a].std()

#计算消费余额的最大、最小值、中位数、方差

b='remainder'
card_feature_f[b+'_max']=card_feature.groupby('id')[b].max()
card_feature_f[b+'_min']=card_feature.groupby('id')[b].min()
card_feature_f[b+'_median']=card_feature.groupby('id')[b].median()
card_feature_f[b+'_std']=card_feature.groupby('id')[b].std()

#计算消费总数

card_feature_f['amount_count']=card_feature.groupby('id')['amount'].count()

In [ ]:
#card_feature_f1.to_csv('input/card_test_feature_f.csv',index='id')

# 第三次处理

In [18]:
card_feature_f=pd.read_csv('input/card_test_feature_f.csv',index_col='id')

In [17]:
#每个id的活跃天数

card_test_date=DataFrame(card_feature.groupby(['id','date'])['id'].count())
card_test_date.to_csv('input/card_test_day.csv',index=True)
card_test_day=pd.read_csv('input/card_test_day.csv')
card_test_day_sum=DataFrame(card_test_day.groupby('id')['id.1'].count())
card_test_day_sum.columns=['card_day_sum']

#每天平均消费额

card_feature_f['card_day_sum']=card_test_day_sum['card_day_sum']
card_feature_f['amount_mean_day']=card_feature_f['amount']/card_feature_f['card_day_sum']

In [18]:
#周末消费均值减去非周末消费均值(每次)

card_feature_f['rd_wd_difference']=card_feature_f['resting_day_amount_mean']-card_feature_f['working_day_amount_mean']


In [19]:
#“圈存转账”“卡充值”“支付领取”三种充值类特征相加

a=card_feature_f['card_recharge_amount_sum'].add(card_feature_f['load_tansfer_amount_sum'],fill_value=0)
card_feature_f['three_recharge_amount_sum']=a.add(card_feature_f['payment_collection_amount_sum'],fill_value=0)

#卡充值总额/活跃天数

card_feature_f['recharge_day_mean']=card_feature_f['three_recharge_amount_sum'].divide(card_feature_f['card_day_sum'],fill_value=0)


In [20]:
#单笔消费额在0-2.5，2.5-5，5-10，10-20的笔数/活跃天数

def get_amount_count(df1,df2):
    a=[2.5,5,10,20]
    for x in range(len(a)):
        if x==0:
            #消费总笔数
            amount_count_x=DataFrame(df1.loc[df1['amount']<=a[x]].groupby('id')['amount'].count())
            amount_count_x.columns=['amount_slice_0_'+str(a[x])+'_count']
            #活跃天数
            amount_day=DataFrame(df1.loc[df1['amount']<=a[x]].groupby(['id','date'])['id'].count())                      
            amount_day.to_csv('input/card_test_amount_day.csv',index=True)
            amount_day=pd.read_csv('input/card_test_amount_day.csv')
            slice_day_sum_x=DataFrame(amount_day.groupby('id')['id.1'].count())
            slice_day_sum_x.columns=['day_sum']
            #df2['amount_slice_0_'+str(a[x])]=amount_count_x['amount'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_0_'+str(a[x])+'_rate_day']=amount_count_x['amount_slice_0_'+str(a[x])+'_count'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_0_'+str(a[x])+'_day_sum']=slice_day_sum_x['day_sum']
            df2=df2.join(amount_count_x)
        else:
            #消费总笔数
            b=df1.loc[df1['amount']>a[x-1]]
            amount_count_x=DataFrame(b.loc[b['amount']<=a[x]].groupby('id')['amount'].count())
            amount_count_x.columns=['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_count']    
            #活跃天数
            amount_day=DataFrame(b.loc[b['amount']<=a[x]].groupby(['id','date'])['id'].count()) 
            amount_day.to_csv('input/card_test_amount_day.csv',index=True)
            amount_day=pd.read_csv('input/card_test_amount_day.csv')
            slice_day_sum_x=DataFrame(amount_day.groupby('id')['id.1'].count())
            slice_day_sum_x.columns=['day_sum']
            #活跃天数
            #df2['amount_slice_'+str(a[x-1])+'_'+str(a[x])]=amount_count_x['amount'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_rate_day']=amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_count'].divide(slice_day_sum_x['day_sum'],fill_value=0)
            amount_count_x['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_day_sum']=slice_day_sum_x['day_sum']
            df2=df2.join(amount_count_x)
            x+=1
    return df2
    #return df2.head()

In [21]:
#运用函数

card_feature_f=get_amount_count(card_feature,card_feature_f)

In [22]:
#单笔消费额在0-2.5，2.5-5，5-10，10-20的活跃笔数占比，活跃天数占比

def get_amount_count1(df):
    a=[0,2.5,5,10,20]
    for x in range(len(a)-1):
        df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_count_rate']=df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_count'].divide(df['amount_count'],fill_value=0)
        df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_day_rate']=df['amount_slice_'+str(a[x])+'_'+str(a[x+1])+'_day_sum'].divide(df['card_day_sum'],fill_value=0)       
    return df 



In [23]:
#运用函数

card_feature_f=get_amount_count1(card_feature_f)

In [24]:
#每天消费总额在0-10，10-20，20-30，30-40,40-50的笔数/活跃天数

def get_amount_count2(df1,df2):
    a=[10,20,30,40,50]
    for x in range(len(a)):
        if x==0:
            #消费笔数
            amount_sum=DataFrame(df1.groupby(['id','date'])['amount'].sum())
            amount_sum.to_csv('input/card_test_amount_sum.csv',index=True)
            amount_sum_x=pd.read_csv('input/card_test_amount_sum.csv')
            slice_sum_x=DataFrame(amount_sum_x.loc[amount_sum_x['amount']<=a[x]].groupby('id')['amount'].count())
            slice_sum_x.columns=['amount_slice_day_0_'+str(a[x])+'_count']
            df2=df2.join(slice_sum_x)
            #df2['amount_slice_0_'+str(a[x])+'_all']=slice_sum_x['amount'].divide(df2['card_day_sum'],fill_value=0)
        else:
            #消费笔数
            amount_sum=DataFrame(df1.groupby(['id','date'])['amount'].sum())
            amount_sum.to_csv('input/card_test_amount_sum.csv',index=True)
            amount_sum_x=pd.read_csv('input/card_test_amount_sum.csv')
            b=amount_sum_x.loc[amount_sum_x['amount']>a[x-1]]
            slice_sum_x=DataFrame(b.loc[b['amount']<=a[x]].groupby('id')['amount'].count())
            slice_sum_x.columns=['amount_slice_day_'+str(a[x-1])+'_'+str(a[x])+'_count']
            df2=df2.join(slice_sum_x)
            #df2['amount_slice_'+str(a[x-1])+'_'+str(a[x])+'_all']=slice_sum_x['amount'].divide(df2['card_day_sum'],fill_value=0)
            x+=1
    return df2

In [25]:
#运用函数

card_feature_f=get_amount_count2(card_feature,card_feature_f)

In [26]:
#每天消费总额在0-10，10-20，20-30，30-40,40-50活跃天数/活跃天数
def get_amount_count3(df):
    a=[0,10,20,30,40,50]
    for x in range(len(a)-1):
        df['amount_slice_day_'+str(a[x])+'_'+str(a[x+1])+'day_rate']=df['amount_slice_day_'+str(a[x])+'_'+str(a[x+1])+'_count'].divide(df['card_day_sum'],fill_value=0)
    return df

In [27]:
#运用函数

card_feature_f=get_amount_count3(card_feature_f)

In [34]:
#导出数据

#card_feature_f.to_csv('input/card_test_feature_ff.csv',index='id')

In [29]:
#导出数据

card_feature_f.to_csv('input/card_test_feature_f_2.csv',index='id')

In [19]:
count_pivot.head()

,POS_recharge,bus,canteen,card_recharge,f_ten_canteen,f_ten_place,hospital,hour_00_01,hour_02_05,hour_06_07,...,other_canteen,other_place,payment_collection,printting,resting_day,shower,summer_vacation,water,winter_vacation,working_day
id,,,,,,,,,,,,,,,,,,,,,
6,864,29,334,18,39,467,0,2,2,2,...,295,83,0,10,204,14,60,349,76,680
7,1253,156,583,6,133,478,0,8,7,40,...,450,236,0,5,362,24,180,313,67,935
12,1005,44,326,20,69,391,0,74,10,22,...,257,342,20,4,236,204,53,134,66,823
13,14,0,7,1,0,6,0,0,0,0,...,7,3,0,0,10,0,16,2,0,6
14,12,2,2,1,1,6,0,0,0,0,...,1,6,0,0,12,1,14,4,0,2


In [28]:
card_feature_f.head()

,POS_recharge_mean,amount_mean,bus_mean,canteen_mean,card_recharge_mean,f_ten_canteen_mean,f_ten_place_mean,hospital_mean,hour_00_01_mean,hour_02_05_mean,...,amount_slice_day_0_10_count,amount_slice_day_10_20_count,amount_slice_day_20_30_count,amount_slice_day_30_40_count,amount_slice_day_40_50_count,amount_slice_day_0_10day_rate,amount_slice_day_10_20day_rate,amount_slice_day_20_30day_rate,amount_slice_day_30_40day_rate,amount_slice_day_40_50day_rate
id,,,,,,,,,,,,,,,,,,,,,
2,0.963087,5.472383,0.013423,0.164430,0.027964,0.011186,0.289709,0.000000,0.111857,0.064877,...,95.0,39.0,22.0,9.0,NaN,0.489691,0.201031,0.113402,0.046392,0.000000
3,1.000000,0.714545,0.000000,0.000000,0.000000,0.000000,0.054545,0.000000,0.072727,0.054545,...,32.0,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000
16,0.956612,6.073037,0.117769,0.033058,0.026860,0.033058,0.477273,0.002066,0.010331,0.000000,...,58.0,19.0,18.0,2.0,5.0,0.495726,0.162393,0.153846,0.017094,0.042735
17,0.951807,5.966867,0.144578,0.000000,0.012048,0.000000,0.240964,0.000000,0.156627,0.000000,...,20.0,3.0,2.0,1.0,NaN,0.714286,0.107143,0.071429,0.035714,0.000000
32,0.917127,5.690304,0.088398,0.505525,0.030387,0.140884,0.234807,0.000000,0.030387,0.005525,...,47.0,20.0,5.0,12.0,3.0,0.470000,0.200000,0.050000,0.120000,0.030000


In [36]:
card_feature_f['amount_slice_day_40_50_count'].head()

id
6      1.0
7      7.0
12    13.0
13     NaN
14     NaN
Name: amount_slice_day_40_50_count, dtype: float64

In [30]:
card_feature_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10771 entries, 2 to 32657
Columns: 336 entries, POS_recharge_mean to amount_slice_day_40_50day_rate
dtypes: float64(302), int64(34)
memory usage: 27.7 MB
